<h1 style="Comic Sans MS; text-align: center; color: #FFC300">Regularized Linear Regression: US demographic and health data</h1>
<h3 id="step1" style="font-family: Comic Sans MS; color: #68FF33">1. Problem statement and data collection</h3>
<p style="color: pink">Sociodemographic and health resource data have been collected by county in the United States and we want to find out if there is any relationship between health resources and sociodemographic data. To do this, we have to:
</p>
<ul style="color: pink">
    <li>set a target variable (health-related) to conduct the analysis.</li>
    <li>perform EDA so we keep the variables that are strictly necessary and eliminate those that are not relevant or do not provide information.</li>
    <li>start solving the problem by implementing a linear regression model and analyze the results. Then, using the same data and default attributes, build a Lasso model and compare the results with the baseline linear regression.</li>
    <li>analyze how R-squared evolves when the hyperparameter of the Lasso model changes (for example we can start testing from a value of 0.0 and work our way up to a value of 20). Draw these values in a line diagram.</li>
    <li>last, after training the Lasso model, if the results are not satisfactory, optimize it using one of the techniques seen in the module.</li>
</ul>

In [1]:
# Your code here

<h3 id="step2" style="font-family: Comic Sans MS; color: #68FF33">2. Exploration and data cleaning</h3>
<p style="color: pink">We will conduct a concise Exploratory Data Analysis (EDA), focusing on essential aspects without delving into intricate details.</p>

<h3 id="step3" style="font-family: Comic Sans MS; color: #68FF33">3. Analysis of univariate variables</h3>
<strong><span style="color: red">Analysis of categorical variables</span></strong>

<strong><span style="color: red">Analysis of numerical variables</span></strong>

<strong><span style="color: red">Correlation Analysis</span></strong>

<h3 id="step4" style="font-family: Comic Sans MS; color: #68FF33">4. Feature engineering</h3>
<strong><span style="color: red">Min-Max scaler</span></strong>

<strong><span style="color: red">Feature selection</span></strong>

<h3 id="step5" style="font-family: Comic Sans MS; color: #68FF33">5. Linear Regression: Model training and optimization</h3>
<strong><span style="color: red">Reading the processed dataset and plotting features</span></strong>

<strong><span style="color: red">Initialization and training of the models</span></strong>